# Project 6 - Pipleline 4 - Predict

In [51]:
import pandas as pd
from os import chdir; chdir('../lib')

from sklearn.preprocessing import StandardScaler
import encoding_module as enc
import database_module as db       
import wiki_module as wiki 

## Mine: Query Wikipedia by page id for content

In [31]:
page_response = wiki.query_page(str('7509291')) #Depth chart

In [52]:
page_response["text"]

u'In sports, a depth chart is used to show the placements of the starting players and the secondary players. Generally a starting player will be listed first or on top while a back-up will be listed below. Depth charts also tend to resemble the actual position locations of certain players. The typical Major League Baseball depth chart consists of a list of players at each position, with the starter or first-string player listed first, followed by replacement and platoon players. For fantasy baseball, typical preseason projection systems such as PECOTA construct depth charts that specify not just the order of the players at each position (starter, replacement or bench player) but also the amount of playing time each person at that position will have. For example, at first base one player may be projected as playing 60 percent of the innings, and another 40 percent for the coming season, while at catcher one player may be projected as playing 80 percent of the innings, and another 20 per

## Parse: Encode page as document vector

In [53]:
from sklearn.externals import joblib
transformer = joblib.load('../data/vectorizer.pkl')

In [54]:
transformer

Pipeline(steps=[('tfidf', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tru...TruncatedSVD(algorithm='randomized', n_components=500, n_iter=10,
       random_state=42, tol=0.0))])

In [55]:
search_vector_doc = enc.get_searchterm_vector(transformer, page_response["text"])

In [58]:
search_vector_doc.values()[0][0][0:15]

array([ 0.08283265,  0.10180993,  0.00833631, -0.14341235, -0.08119117,
        0.01505812, -0.00918974, -0.01755578, -0.00097451,  0.03420582,
        0.016862  ,  0.01977553,  0.02640352,  0.003259  , -0.0140419 ])

## Present: Load previoly trained and pickled model

In [59]:
from sklearn.naive_bayes import GaussianNB
this_naive_calssifier = GaussianNB()
this_naive_calssifier = joblib.load('project6_pickled_model.pkl')

In [60]:
this_naive_calssifier

GaussianNB(priors=None)

## Model: Predict category and display category and prediction confidence

In [61]:
predicted = this_naive_calssifier.predict(search_vector_doc.values()[0])

In [62]:
predicted

array([2835113], dtype=int64)

In [63]:
SQLstr = """

select category_id, category_name from category where category_id = 2835113
"""
category = db.select_statement(SQLstr, location="remote")

In [64]:
category

,category_id,category_name
0,2835113,shoes


In [65]:
probs = this_naive_calssifier.predict_proba(search_vector_doc.values()[0])

In [66]:
probs

array([[  4.51758837e-13,   0.00000000e+00,   6.68274166e-07,
          1.76265723e-14,   2.20147913e-22,   8.13693997e-22,
          5.25410521e-23,   2.10090916e-12,   1.93025247e-19,
          2.02722781e-19,   4.31389630e-13,   9.99999332e-01,
          3.72462151e-31,   1.75826469e-18,   1.00688030e-11,
          7.02898758e-11,   1.77494847e-22]])